In [1]:
from hello.hello3 import *
from officelib.xllib import *
from officelib import const

In [42]:
from time import sleep
def set_mg(sp):
    try:
        app.setmg(1, sp)
    except HelloError:
        app.login()
        app.setmg(1, sp)
        
def update_ss(cell, val):
    cell.Value = val
    c = cell.Offset(2, 1)
    c.Select()
    return c
        
def frange(start, stop, step):
    i = 0
    v = start + i * step
    while v < stop:
        yield v
        i += 1
        v = start + i * step
        

def run_test(name, mfc_max, c=None):
    tmin = mfc_max / 10
    print("Testing ", name, "MFC: %.1f LPM Max" % mfc_max)
    app.login()
    for fr in frange(tmin, mfc_max+tmin/2, tmin):
        set_mg(fr)
        v = input("%.2f LPM:" % fr)
        if v.lower() in ("break", "quit", "q", "stop"):
            break
        c = update_ss(c, v)

def air_on():
    app.setph(2,0,0)
    app.setdo(2,0,0)
    
def n2_on():
    app.setph(2,0,0)
    app.setdo(1, 100, 0)
    
def o2_on():
    app.setph(2,0,0)
    app.setdo(1, 0, 100)
    
def co2_on():
    app.setconfig("pH", "CO2_Manual_Max_(%)", 100)
    app.setph(1, 100, 0)
    app.setdo(2, 0, 0)
    

cc = {
    "co2": 4,
    "air": 1,
    "n2": 2,
    "o2": 3
}

on_fn = {
    "co2": co2_on,
    "air": air_on,
    "n2": n2_on,
    "o2": o2_on
}

    
def s():
    return xl.Selection

def run(name, mfcm):
    c = cc[name]
    app.login()
    if name == "co2":
        co2m = app.getConfig()['pH']['CO2_Manual_Max_(%)']
    on_fn[name]()
    run_test(name, mfcm, cells(2,c))
    if name == "co2":
        app.login()
        app.setconfig("pH", "CO2_Manual_Max_(%)", co2m)

def all_off():
    app.login()
    app.setag(0, 12)
    app.setpumpa(0, 0)
    app.setpumpb(0, 0)
    app.setpumpc(0, 0)
    app.setpumpsample(0, 0)
    
def connect():
    global app
    ip = input("Enter IP Address: ")
    app = HelloApp(ip)
    
def init():
    global xl, wb, ws, cells, cell_range
    xl, wb, ws, cells = xlObjs()
    cell_range = cells.Range
    cell_range("A1:D1").Value = [("Air", "N2", "O2", "CO2")]
    

In [12]:
init()
connect()
all_off()

Enter IP Address: 192.168.1.20


In [15]:
run("air", 10)

Testing  air MFC: 10.0 LPM Max
1.00 LPM:1.08
2.00 LPM:2.12
3.00 LPM:3.20
4.00 LPM:4.25
5.00 LPM:5.29
6.00 LPM:6.34
7.00 LPM:7.39
8.00 LPM:8.43
9.00 LPM:9.45
10.00 LPM:10.50


In [16]:
run("n2", 10)

Testing  n2 MFC: 10.0 LPM Max
1.00 LPM:1.06
2.00 LPM:2.13
3.00 LPM:3.20
4.00 LPM:4.27
5.00 LPM:5.33
6.00 LPM:6.38
7.00 LPM:7.42
8.00 LPM:8.48
9.00 LPM:9.53
10.00 LPM:10.59


In [22]:
run("o2", 10)

Testing  o2 MFC: 10.0 LPM Max
1.00 LPM:1.07
2.00 LPM:2.108
3.00 LPM:3.10
4.00 LPM:4.25
5.00 LPM:5.17
6.00 LPM:6.20
7.00 LPM:7.22
8.00 LPM:8.23
9.00 LPM:9.26
10.00 LPM:10.26


In [44]:
run("co2", 2)

Testing  co2 MFC: 2.0 LPM Max
0.20 LPM:.222
0.40 LPM:.427
0.60 LPM:.634
0.80 LPM:.841
1.00 LPM:1.045
1.20 LPM:1.248
1.40 LPM:1.452
1.60 LPM:1.656
1.80 LPM:1.861
2.00 LPM:2.058
